In [1]:
class Passenger:
    def __init__(self, group_index, passenger_id, description, transit_time=0):
        """
            group_index : 
            passenger_id : 
            description : men, women, wheelchair, child
            business_economic : class (Business, Economic)
            transit_time : optional parameter (inf if there is not transit time)
        
        """
        self.group_index = group_index
        self.passenger_id = passenger_id
        self.description = description
        # self.business_economic = business_economic
        self.transit_time = transit_time
        self.weight()
    
    def weight(self):
        if self.description == 'women':
            self.weight = 70
        elif self.description == 'men':
            self.weight = 85
        elif self.description == 'child':
            self.weight = 35
        else: 
            self.weight = 70 # need to check this value

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
passenger_id = 0

In [5]:
class Group:

    def __init__(self, id_group, num_women, num_men, transit_time=0):
        self.id_group = id_group
        self.num_women = num_women
        self.num_men = num_men
        self.transit_time = transit_time

        self.members = []

        global passenger_id

        for i in range(self.num_women):
            self.members.append(Passenger(
                self.id_group,
                passenger_id,
                'women',
                self.transit_time
            ))
            passenger_id+=1
        
        for i in range(self.num_men):
            self.members.append(Passenger(
                self.id_group,
                passenger_id,
                'men',
                self.transit_time
            ))
            passenger_id+=1
        
        return
    
    
        

In [6]:
def read_and_preprocess(file, date):
    df = pd.read_excel(file, date)
    
    df['TransitTime (min)'] = df['TransitTime'].apply(lambda x: float(str(x).split(':')[0])*60+float(str(x).split(':')[1]))

    df[['Femmes', 'Hommes', 'Enfants', 'WCHR', 'WCHB']] = df[['Femmes', 'Hommes', 'Enfants', 'WCHR', 'WCHB']].fillna(0)

    return df

def get_list_passengers(df):
    list_groups = {}

    for idx, row in df.iterrows():

        list_groups[int(idx)] = Group(
            id_group=int(idx),
            num_women=int(row['Femmes']),
            num_men=int(row['Hommes']),
            transit_time=row['TransitTime (min)']
        )
    
    return list_groups

In [9]:
df = read_and_preprocess('DataSeating - V3.xlsx', '21Oct')

In [10]:
df.head()

,Numéro du groupe,Femmes,Hommes,Enfants,WCHR,WCHB,Classe,TransitTime,TransitTime (min)
0,1,0.0,1.0,0.0,0.0,0.0,Y,01:15:00,75.0
1,2,1.0,0.0,0.0,0.0,0.0,Y,00:00:00,0.0
2,3,0.0,1.0,0.0,0.0,0.0,Y,00:00:00,0.0
3,4,1.0,0.0,0.0,0.0,0.0,Y,00:00:00,0.0
4,5,0.0,2.0,0.0,0.0,0.0,Y,05:25:00,325.0


In [11]:
groups_list = get_list_passengers(df)

In [12]:
passengers_list = list()

for g in groups_list.keys():
    passengers_list += groups_list[g].members

In [13]:
len(passengers_list)

151

In [14]:
passengers_list

# Creating the Airplane

In [15]:
airplane = dict()

airplane['xmax'] = 28
airplane['ymax'] = 7
airplane['corridor_y'] = 4

airplane['seats'] = dict()

# considering there is only business class
real_seats = []
corridor_seats = []
for x in range(1, airplane['xmax']+1):
    for y  in range(1, airplane['ymax']+1):
        if y != airplane['corridor_y']:
            real_seats.append([x, y])
        else:
            corridor_seats.append([x, y])

airplane['seats']['real'] = real_seats
airplane['seats']['corridor'] = corridor_seats

del real_seats, corridor_seats

print(airplane)



{'xmax': 28, 'ymax': 7, 'corridor_y': 4, 'seats': {'real': [[1, 1], [1, 2], [1, 3], [1, 5], [1, 6], [1, 7], [2, 1], [2, 2], [2, 3], [2, 5], [2, 6], [2, 7], [3, 1], [3, 2], [3, 3], [3, 5], [3, 6], [3, 7], [4, 1], [4, 2], [4, 3], [4, 5], [4, 6], [4, 7], [5, 1], [5, 2], [5, 3], [5, 5], [5, 6], [5, 7], [6, 1], [6, 2], [6, 3], [6, 5], [6, 6], [6, 7], [7, 1], [7, 2], [7, 3], [7, 5], [7, 6], [7, 7], [8, 1], [8, 2], [8, 3], [8, 5], [8, 6], [8, 7], [9, 1], [9, 2], [9, 3], [9, 5], [9, 6], [9, 7], [10, 1], [10, 2], [10, 3], [10, 5], [10, 6], [10, 7], [11, 1], [11, 2], [11, 3], [11, 5], [11, 6], [11, 7], [12, 1], [12, 2], [12, 3], [12, 5], [12, 6], [12, 7], [13, 1], [13, 2], [13, 3], [13, 5], [13, 6], [13, 7], [14, 1], [14, 2], [14, 3], [14, 5], [14, 6], [14, 7], [15, 1], [15, 2], [15, 3], [15, 5], [15, 6], [15, 7], [16, 1], [16, 2], [16, 3], [16, 5], [16, 6], [16, 7], [17, 1], [17, 2], [17, 3], [17, 5], [17, 6], [17, 7], [18, 1], [18, 2], [18, 3], [18, 5], [18, 6], [18, 7], [19, 1], [19, 2], [19,

# Defining the model

In [16]:
from gurobipy import *

In [17]:
m = Model('First Try')

Restricted license - for non-production use only - expires 2023-10-25


## Decision Variables

In [18]:
A = dict()

# todo - add zeroes on the border of the plane using extreme As defined as zeroes by constraints
for x in range(airplane['xmax']+1):
    for y in range(airplane['ymax']+1):
        for p in passengers_list:
            A[x, y, p.passenger_id] = m.addVar(vtype=GRB.BINARY, name="A_{},{},{}".format(x,y,p.passenger_id))

## Constraints

### Barycenter

In [19]:
# Barycenter: x \in [13,17], y \in [3,5]
total_weight_passengers = sum(p.weight for p in passengers_list)

barycenter = dict()

barycenter['xinf'] = m.addConstr(quicksum(p.weight*x*A[x, y, p.passenger_id] for x,y in airplane['seats']['real'] for p in passengers_list) >= 13*total_weight_passengers, name="barycenter_x_inf")
barycenter['xsup'] = m.addConstr(quicksum(p.weight*x*A[x, y, p.passenger_id] for x,y in airplane['seats']['real'] for p in passengers_list) <= 17*total_weight_passengers, name="barycenter_x_sup")
barycenter['yinf'] = m.addConstr(quicksum(p.weight*x*A[x, y, p.passenger_id] for x,y in airplane['seats']['real'] for p in passengers_list) >= 3*total_weight_passengers, name="barycenter_y_inf")
barycenter['ysup'] = m.addConstr(quicksum(p.weight*x*A[x, y, p.passenger_id] for x,y in airplane['seats']['real'] for p in passengers_list) <= 5*total_weight_passengers, name="barycenter_y_sup")


### No passenger in the corridor

In [20]:
corridor = m.addConstr(quicksum(A[x,y,p.passenger_id] for x, y in airplane['seats']['corridor'] for p in passengers_list) == 0)

### At most one passenger per seat

In [21]:
passenger_seat_constr = dict()

for x, y in airplane['seats']['real']:
    passenger_seat_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] for p in passengers_list) <= 1, name='1 passenger at most at seat {},{}'.format(x,y))


### One seat per passenger

In [22]:
seat_passenger_constr = dict()

for p in passengers_list:
    seat_passenger_constr[p] = m.addConstr(quicksum(A[x, y, p.passenger_id] for x, y in airplane['seats']['real']) == 1, name='1 seat per passenger {}'.format(p.passenger_id))

## Satisfaction variables

In [35]:
S = dict()
is_pair_y_the_same_group = dict() 
is_pair_x_the_same_group = dict()
aux1 = dict()
aux2 = dict()
c1 = dict()
c2 = dict()

for x, y in airplane['seats']['real']:
    S[(x,y)] = m.addVar(vtype=GRB.CONTINUOUS, name="satisfaction_{}_{}".format(x, y))
    aux1 [(x,y)] = m.addVar(vtype=GRB.BINARY, name="aux1_{}_{}".format(x, y))
    aux2 [(x,y)] = m.addVar(vtype=GRB.BINARY, name="aux2_{}_{}".format(x, y))
    c1 [(x,y)] = m.addVar(vtype=GRB.BINARY, name="c1_{}_{}".format(x, y))
    c2 [(x,y)] = m.addVar(vtype=GRB.BINARY, name="c2_{}_{}".format(x, y))
    
    if y != 7:
        is_pair_y_the_same_group [(x,y)] = m.addVar(vtype=GRB.BINARY, name="y_pair_{}_{}".format(x, y))
    
    if x != 28:
        is_pair_x_the_same_group [(x,y)] = m.addVar(vtype=GRB.BINARY, name="x_pair_{}_{}".format(x, y))


## Objective function

In [36]:
m.setObjective(quicksum(S[(x,y)] for x, y in airplane['seats']['real']), GRB.MAXIMIZE)

### constraints for the satisfaction levels

In [39]:
s_passenger_constr = dict()
is_pair_y1_the_same_group_constr = dict()
is_pair_x1_the_same_group_constr = dict()
is_pair_y2_the_same_group_constr = dict()
is_pair_x2_the_same_group_constr = dict()
is_pair_y3_the_same_group_constr = dict()
is_pair_x3_the_same_group_constr = dict()
is_pair_y4_the_same_group_constr = dict()
is_pair_x4_the_same_group_constr = dict()
is_pair_y5_the_same_group_constr = dict()
is_pair_x5_the_same_group_constr = dict()
is_pair_y6_the_same_group_constr = dict()
is_pair_x6_the_same_group_constr = dict()
is_pair_y7_the_same_group_constr = dict()
is_pair_x7_the_same_group_constr = dict()

cy1_constr = dict()
cy2_constr = dict()
cx1_constr = dict()
cx2_constr = dict()


for x, y in airplane['seats']['real']:
    if y != 7 and x != 28 and y != 3:
        
        # if (x,y) and (x,y+1) are from the same group and are occupied, then is_pair_y_the_same_group(x,y) = 1 and is 0 otherwise
        is_pair_y1_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y+1, p.passenger_id] * p.group_index for p in passengers_list) + 0.5 <= 100 * aux1[(x,y)], name='pair y definition1 {}_{}'.format(x, y))
        is_pair_y2_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y+1, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - 0.5 <= 100 *(1 - aux1[(x,y)]), name='pair y definition2 {}_{}'.format(x, y))
        is_pair_y4_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y+1, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + 0.5 <= 100 * aux2[(x,y)], name='pair y definition4 {}_{}'.format(x, y))
        is_pair_y5_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y+1, p.passenger_id] * p.group_index for p in passengers_list) - 0.5 <= 100 *(1 - aux2[(x,y)]), name='pair y definition5 {}_{}'.format(x, y))
        is_pair_y6_the_same_group_constr[(x,y)] = m.addConstr(aux1[(x,y)] + aux2[(x,y)] - 1 <= is_pair_y_the_same_group[(x,y)] + 100*(1 - c1[(x,y)]), name='pair y definition6 {}_{}'.format(x, y))
        is_pair_y7_the_same_group_constr[(x,y)] = m.addConstr(c1[(x,y)] >= is_pair_y_the_same_group[(x,y)], name='pair y definition7 {}_{}'.format(x, y))
        cy1_constr[(x,y)] = m.addConstr(100*quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + 100*quicksum(A[x, y+1, p.passenger_id] * p.group_index for p in passengers_list)  >= c1[(x,y)], name='cy definition1 {}_{}'.format(x, y))
        cy2_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + quicksum(A[x, y+1, p.passenger_id] * p.group_index for p in passengers_list)  <= 100*c1[(x,y)], name='cy definition2 {}_{}'.format(x, y))

        # if (x,y) and (x+1,y) are from the same group and are occupied, then is_pair_x_the_same_group(x,y) = 1 and is 0 otherwise
        is_pair_x1_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x+1, y, p.passenger_id] * p.group_index for p in passengers_list) + 0.5 <= 100 * aux1[(x,y)], name='pair x definition1 {}_{}'.format(x, y))
        is_pair_x2_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x+1, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - 0.5 <= 100 *(1 - aux1[(x,y)]), name='pair x definition2 {}_{}'.format(x, y))
        is_pair_x4_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x+1, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + 0.5 <= 100 * aux2[(x,y)], name='pair x definition4 {}_{}'.format(x, y))
        is_pair_x5_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x+1, y, p.passenger_id] * p.group_index for p in passengers_list) - 0.5 <= 100 *(1 - aux2[(x,y)]), name='pair x definition5 {}_{}'.format(x, y))
        is_pair_y6_the_same_group_constr[(x,y)] = m.addConstr(aux1[(x,y)] + aux2[(x,y)] - 1 <= is_pair_x_the_same_group[(x,y)] + 100*(1 - c2[(x,y)]), name='pair x definition6 {}_{}'.format(x, y))
        is_pair_x7_the_same_group_constr[(x,y)] = m.addConstr(c2[(x,y)] >= is_pair_x_the_same_group[(x,y)], name='pair x definition7 {}_{}'.format(x, y))
        cx1_constr[(x,y)] = m.addConstr(100*quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + 100*quicksum(A[x+1, y, p.passenger_id] * p.group_index for p in passengers_list)  >= c2[(x,y)], name='cx definition1 {}_{}'.format(x, y))
        cx2_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + quicksum(A[x+1, y, p.passenger_id] * p.group_index for p in passengers_list)  <= 100*c2[(x,y)], name='cx definition2 {}_{}'.format(x, y))


    elif y == 3 and x != 28:
        # if (x,y) and (x,y+1) are separated by a corridor
        is_pair_y1_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y+2, p.passenger_id] * p.group_index for p in passengers_list) + 0.5 <= 100 * aux1[(x,y)], name='pair y definition1 {}_{}'.format(x, y))
        is_pair_y2_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y+2, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - 0.5 <= 100 *(1 - aux1[(x,y)]), name='pair y definition2 {}_{}'.format(x, y))
        is_pair_y4_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y+2, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + 0.5 <= 100 * aux2[(x,y)], name='pair y definition4 {}_{}'.format(x, y))
        is_pair_y5_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y+2, p.passenger_id] * p.group_index for p in passengers_list) - 0.5 <= 100 *(1 - aux2[(x,y)]), name='pair y definition5 {}_{}'.format(x, y))
        is_pair_y6_the_same_group_constr[(x,y)] = m.addConstr(aux1[(x,y)] + aux2[(x,y)] - 1 <= is_pair_y_the_same_group[(x,y)] + 100*(1 - c1[(x,y)]), name='pair y definition6 {}_{}'.format(x, y))
        is_pair_y7_the_same_group_constr[(x,y)] = m.addConstr(c1[(x,y)] >= is_pair_y_the_same_group[(x,y)], name='pair y definition7 {}_{}'.format(x, y))
        cy1_constr[(x,y)] = m.addConstr(100*quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + 100*quicksum(A[x, y+2, p.passenger_id] * p.group_index for p in passengers_list)  >= c1[(x,y)], name='cy definition1 {}_{}'.format(x, y))
        cy2_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + quicksum(A[x, y+2, p.passenger_id] * p.group_index for p in passengers_list)  <= 100*c1[(x,y)], name='cy definition2 {}_{}'.format(x, y))

        
for x, y in airplane['seats']['real']:
    if y == 1:
        if x == 1:
            s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + 0.5*(is_pair_x_the_same_group[(x,y)]))
        else:
            if x == 28:
                s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + 0.5*(is_pair_x_the_same_group[(x-1,y)]))
            else:
                s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + 0.5*(is_pair_x_the_same_group[(x,y)] + is_pair_x_the_same_group[(x-1,y)]))
    else:
        if x == 1:
            if y == 7:
                s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y-1)] + 0.5*(is_pair_x_the_same_group[(x,y)] ))
            else:
                if y == 5:
                    s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + is_pair_y_the_same_group[(x,y-2)] + 0.5*(is_pair_x_the_same_group[(x,y)] ))
                else:
                    s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + is_pair_y_the_same_group[(x,y-1)] + 0.5*(is_pair_x_the_same_group[(x,y)] ))
        else:
            if y == 7:
                if x == 28:
                    s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y-1)] + 0.5*( is_pair_x_the_same_group[(x-1,y)]))
                else:
                    s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y-1)] + 0.5*(is_pair_x_the_same_group[(x,y)] + is_pair_x_the_same_group[(x-1,y)]))
            else:
                if x == 28:
                    if y == 5:
                        s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + is_pair_y_the_same_group[(x,y-2)] + 0.5*( is_pair_x_the_same_group[(x-1,y)]))
                    else:
                        s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + is_pair_y_the_same_group[(x,y-1)] + 0.5*( is_pair_x_the_same_group[(x-1,y)]))
                else:
                    if y == 5:
                        s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + is_pair_y_the_same_group[(x,y-2)] + 0.5*(is_pair_x_the_same_group[(x,y)] + is_pair_x_the_same_group[(x-1,y)]))
                    else:
                        s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + is_pair_y_the_same_group[(x,y-1)] + 0.5*(is_pair_x_the_same_group[(x,y)] + is_pair_x_the_same_group[(x-1,y)]))
        
    

In [40]:
m.update()

m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license